In [1]:
import numpy as np
import pandas as pd
import json
from tqdm import tqdm
import os

In [2]:
file = open("/research/video_metaphor/LLaVA/results/llava13-b/vmcd_ispy/24f_4parts/5_ep/answers_t_0_2_test.jsonl", "r")
questions = [json.loads(q) for q in file]
print(len(questions))

750


In [13]:
true_captions = [ q["true_caption"] for q in questions]
pred_captions = [ q["text"].split("\n")[0] for q in questions]
video_ids = [ q["video_id"] for q in questions]

In [14]:
predicted_df = pd.DataFrame({"video_id": video_ids, "true_caption":true_captions, "pred_captions": pred_captions})
reference_df = predicted_df.groupby('video_id')['true_caption'].apply(list).reset_index(name="reference_captions")
merged_df = pd.merge(predicted_df, reference_df, on=['video_id'])
reference_captions = merged_df["reference_captions"].tolist()
merged_df.head()

,video_id,true_caption,pred_captions,reference_captions
0,jmNrvlQ73pU,the burger is as comforting as relationship,the toys are as cute as the girl.,"[the burger is as comforting as relationship, ..."
1,jmNrvlQ73pU,a burger is as much a source of satisfaction a...,the toy is as cute as the girl.,"[the burger is as comforting as relationship, ..."
2,jmNrvlQ73pU,the burger is as a bond as kids relationship,the chocolate is as sweet as the relationship ...,"[the burger is as comforting as relationship, ..."
3,D93S3UVU8pk,the game is as fun as the cartoon,the drink is as cool as a polar bear.,"[the game is as fun as the cartoon, the game i..."
4,D93S3UVU8pk,the game is as interesting as a cartoon.,the game is as exciting as a superhero.,"[the game is as fun as the cartoon, the game i..."


In [15]:
merged_df = merged_df.drop_duplicates(["video_id"], keep="last")
print(len(merged_df))
merged_df.head()

250


,video_id,true_caption,pred_captions,reference_captions
2,jmNrvlQ73pU,the burger is as a bond as kids relationship,the chocolate is as sweet as the relationship ...,"[the burger is as comforting as relationship, ..."
5,D93S3UVU8pk,the game is as enjoyable as the cartoon,the game is as intense as a battlefield.,"[the game is as fun as the cartoon, the game i..."
8,jcC3xM8Wa1w,the cookie cutters are as versatile as the art...,the chocolate is as delicious as the cookie.,[cookies are as delicious as the ways to eat t...
11,IQovoot_ZUM,a refreshing beverage is as unifying as indivi...,the drink is as cool as a robot.,[a refreshing beverage is as bonding as indivi...
14,TL4LEfRr1TI,the shoes are as exciting as a football game i...,the drink is as intense as a fight.,[shoes are as goal oriented as a ball moving t...


In [16]:
true_captions = merged_df["reference_captions"].tolist()
pred_captions = merged_df["pred_captions"].tolist()
video_ids = merged_df["video_id"].tolist()

In [17]:
ground_truth1 = [cap_list[0] for cap_list in true_captions]
ground_truth2 = [cap_list[1] for cap_list in true_captions]
ground_truth3 = [cap_list[2] for cap_list in true_captions]

In [18]:
final_df = pd.DataFrame({"video_id": video_ids, "ground_truth1":ground_truth1, "ground_truth2":ground_truth2,"ground_truth3":ground_truth3,"pred_captions": pred_captions})
print(len(final_df))
final_df.head()

250


,video_id,ground_truth1,ground_truth2,ground_truth3,pred_captions
0,jmNrvlQ73pU,the burger is as comforting as relationship,a burger is as much a source of satisfaction a...,the burger is as a bond as kids relationship,the chocolate is as sweet as the relationship ...
1,D93S3UVU8pk,the game is as fun as the cartoon,the game is as interesting as a cartoon.,the game is as enjoyable as the cartoon,the game is as intense as a battlefield.
2,jcC3xM8Wa1w,cookies are as delicious as the ways to eat them,cookies are as delicious as their variety,the cookie cutters are as versatile as the art...,the chocolate is as delicious as the cookie.
3,IQovoot_ZUM,a refreshing beverage is as bonding as individ...,a cool drink is as connecting as people.,a refreshing beverage is as unifying as indivi...,the drink is as cool as a robot.
4,TL4LEfRr1TI,shoes are as goal oriented as a ball moving to...,shoes are as charged up as the football players,the shoes are as exciting as a football game i...,the drink is as intense as a fight.


In [19]:
final_df.to_csv("/research/video_metaphor/LLaVA/results/llava13-b/vmcd_ispy/24f_4parts/5_ep/answers_t_0_2_test.csv", index=False)